In [1]:
from deepagents import create_deep_agent
from langchain_openai import ChatOpenAI
from langchain_core.tools import tool
import yfinance as yf
import logging
from typing import Dict
import json

In [2]:
import os
os.environ["OPENAI_API_KEY"] = os.getenv('OPENAI_API_KEY')

In [3]:
llm = ChatOpenAI(model="gpt-4.1-2025-04-14", temperature=0.0)

In [4]:
@tool
def get_stock_price(symbol: str) -> str:
    """Get current stock price and basic information."""
    logging.info(f"[TOOL] Fetching stock price for: {symbol}")
    try:
        stock = yf.Ticker(symbol)
        info = stock.info
        hist = stock.history(period="1d")
        if hist.empty:
            logging.error("No historical data found")
            return json.dumps({"error": f"Could not retrieve data for {symbol}"})
            
        current_price = hist['Close'].iloc[-1]
        result = {
            "symbol": symbol,
            "current_price": round(current_price, 2),
            "company_name": info.get('longName', symbol),
            "market_cap": info.get('marketCap', 0),
            "pe_ratio": info.get('trailingPE', 'N/A'),
            "52_week_high": info.get('fiftyTwoWeekHigh', 0),
            "52_week_low": info.get('fiftyTwoWeekLow', 0)
        }
        logging.info(f"[TOOL RESULT] {result}")
        return json.dumps(result, indent=2)

    except Exception as e:
        logging.exception("Exception in get_stock_price")
        return json.dumps({"error": str(e)})


In [5]:
@tool
def get_financial_statements(symbol: str) -> str:
    """Retrieve key financial statement data."""
    try:
        stock = yf.Ticker(symbol)
        financials = stock.financials
        balance_sheet = stock.balance_sheet
        
        latest_year = financials.columns[0]
        
        return json.dumps({
            "symbol": symbol,
            "period": str(latest_year.year),
            "revenue": float(financials.loc['Total Revenue', latest_year]) if 'Total Revenue' in financials.index else 'N/A',
            "net_income": float(financials.loc['Net Income', latest_year]) if 'Net Income' in financials.index else 'N/A',
            "total_assets": float(balance_sheet.loc['Total Assets', latest_year]) if 'Total Assets' in balance_sheet.index else 'N/A',
            "total_debt": float(balance_sheet.loc['Total Debt', latest_year]) if 'Total Debt' in balance_sheet.index else 'N/A'
        }, indent=2)
    except Exception as e:
        return f"Error: {str(e)}"

In [6]:
@tool
def get_technical_indicators(symbol: str, period: str = "3mo") -> str:
    """Calculate key technical indicators."""
    try:
        stock = yf.Ticker(symbol)
        hist = stock.history(period=period)
        
        if hist.empty:
            return f"Error: No historical data for {symbol}"
        
        hist['SMA_20'] = hist['Close'].rolling(window=20).mean()
        hist['SMA_50'] = hist['Close'].rolling(window=50).mean()
        
        delta = hist['Close'].diff()
        gain = (delta.where(delta > 0, 0)).rolling(window=14).mean()
        loss = (-delta.where(delta < 0, 0)).rolling(window=14).mean()
        rs = gain / loss
        rsi = 100 - (100 / (1 + rs))
        
        latest = hist.iloc[-1]
        latest_rsi = rsi.iloc[-1]
        
        return json.dumps({
            "symbol": symbol,
            "current_price": round(latest['Close'], 2),
            "sma_20": round(latest['SMA_20'], 2),
            "sma_50": round(latest['SMA_50'], 2),
            "rsi": round(latest_rsi, 2),
            "volume": int(latest['Volume']),
            "trend_signal": "bullish" if latest['Close'] > latest['SMA_20'] > latest['SMA_50'] else "bearish"
        }, indent=2)
    except Exception as e:
        return f"Error: {str(e)}"

In [7]:
# Sub-agent configurations
fundamental_analyst = {
    "name": "fundamental-analyst",
    "description": "Performs deep fundamental analysis of companies including financial ratios, growth metrics, and valuation",
    "prompt": """You are an expert fundamental analyst with 15+ years of experience. 
    Focus on:
    - Financial statement analysis
    - Ratio analysis (P/E, P/B, ROE, ROA, Debt-to-Equity)
    - Growth metrics and trends
    - Industry comparisons
    - Intrinsic value calculations
    Always provide specific numbers and cite your sources."""
}

technical_analyst = {
    "name": "technical-analyst", 
    "description": "Analyzes price patterns, technical indicators, and trading signals",
    "prompt": """You are a professional technical analyst specializing in chart analysis and trading signals.
    Focus on:
    - Price action and trend analysis
    - Technical indicators (RSI, MACD, Moving Averages)
    - Support and resistance levels
    - Volume analysis
    - Entry/exit recommendations
    Provide specific price levels and timeframes for your recommendations."""
}

risk_analyst = {
    "name": "risk-analyst",
    "description": "Evaluates investment risks and provides risk assessment",
    "prompt": """You are a risk management specialist focused on identifying and quantifying investment risks.
    Focus on:
    - Market risk analysis
    - Company-specific risks
    - Sector and industry risks
    - Liquidity and credit risks
    - Regulatory and compliance risks
    Always quantify risks where possible and suggest mitigation strategies."""
}

subagents = [fundamental_analyst, technical_analyst, risk_analyst]

In [8]:
# Main research instructions
research_instructions = """You are an elite stock research analyst with access to multiple specialized tools and sub-agents. 

Your research process should be systematic and comprehensive:

1. **Initial Data Gathering**: Start by collecting basic stock information, price data, and recent news
2. **Fundamental Analysis**: Deep dive into financial statements, ratios, and company fundamentals
3. **Technical Analysis**: Analyze price patterns, trends, and technical indicators
4. **Risk Assessment**: Identify and evaluate potential risks
5. **Competitive Analysis**: Compare with industry peers when relevant
6. **Synthesis**: Combine all findings into a coherent investment thesis
7. **Recommendation**: Provide clear buy/sell/hold recommendation with price targets

Always:
- Use specific data and numbers to support your analysis
- Cite your sources and methodology
- Consider multiple perspectives and potential scenarios
- Provide actionable insights and concrete recommendations
- Structure your final report professionally

When using sub-agents, provide them with specific, focused tasks and incorporate their specialized insights into your overall analysis."""

# Define all tools
tools = [
    get_stock_price,
    get_financial_statements, 
    get_technical_indicators
]

# Create the DeepAgent
stock_research_agent = create_deep_agent(
    tools=tools,
    instructions=research_instructions,
    subagents=subagents,
    model=llm
)

In [9]:
def run_stock_research(query: str):
    """Run the stock research agent and return the final message content with debug logging."""
    try:
        logging.info(f"[run_stock_research] Query received: {query}")
        
        result = stock_research_agent.invoke({
            "messages": [{"role": "user", "content": query}]
        })

        logging.debug(f"[run_stock_research] Full result: {result}")
        
        messages = result.get("messages", [])
        output_text = ""
        
        if not messages:
            logging.warning("[run_stock_research] No messages returned in result.")
            output_text = "Error: No response messages received."
        elif isinstance(messages[-1], dict):
            output_text = messages[-1].get("content", "")
            logging.debug(f"[run_stock_research] Output content from dict: {output_text}")
        elif hasattr(messages[-1], "content"):
            output_text = messages[-1].content
            logging.debug(f"[run_stock_research] Output content from object: {output_text}")
        else:
            logging.error("[run_stock_research] Unrecognized message format.")
            output_text = "Error: Invalid response message format."

        file_output = ""
        if "files" in result:
            file_output += "\n\n=== Generated Research Files ===\n"
            for filename, content in result["files"].items():
                preview = content[:500] + "..." if len(content) > 500 else content
                file_output += f"\n**{filename}**\n{preview}\n"
                logging.debug(f"[run_stock_research] File: {filename}, Preview: {preview[:100]}")

        return output_text + file_output

    except Exception as e:
        logging.exception("[run_stock_research] Exception during invocation:")
        return f"Error: {str(e)}"

In [10]:
run_stock_research("Comprehensive analysis on 삼성전자 answer korean. (005930.KS)")

'# 삼성전자(005930.KS) 종합 분석 리포트\n\n---\n\n## 1. 기본 정보 및 최근 동향\n\n- **현재가**: 70,100원 (2024.6 기준)\n- **시가총액**: 약 462조 원\n- **52주 최고/최저**: 74,000원 / 49,900원\n- **PER**: N/A (2023년 실적 저하로 인한 일시적 미표시)\n- **주요 사업**: 반도체(메모리, 파운드리), 스마트폰, 가전, 디스플레이 등\n\n### 최근 주요 뉴스 요약 (2024년 상반기 기준)\n- 메모리 반도체 가격 반등, AI 서버·데이터센터 수요 급증\n- HBM(고대역폭 메모리) 공급 확대, 글로벌 AI 기업향 공급 본격화\n- 1분기 영업이익 6.6조원(전년 동기 대비 10배 증가), 메모리 부문 흑자 전환\n- 미국 텍사스 파운드리 투자(170억 달러 규모) 진행, 하반기 가동 목표\n- 갤럭시 S24 등 신제품 출시, AI 기능 강화로 모바일 부문 실적 개선 기대\n- SK하이닉스와 HBM 경쟁 심화, 중국발 공급과잉 우려 등 리스크 존재\n\n---\n\n## 2. 재무제표 및 주요 재무지표 분석\n\n- **2024년(연간) 매출**: 300.9조 원\n- **2024년(연간) 순이익**: 3.36조 원 (2023년 대비 대폭 개선)\n- **총자산**: 514.5조 원\n- **총부채**: 19.3조 원 (부채비율 약 3.8%로 매우 안정적)\n- **영업이익률**: 약 2.2% (2023년 저점에서 2024년 빠른 회복세)\n- **현금흐름**: 대규모 투자 지속(파운드리, HBM 등), 현금성 자산 풍부\n\n---\n\n## 3. 기술적 분석\n\n- **현재가**: 70,100원\n- **20일 이동평균선(SMA20)**: 70,300원 (단기 횡보)\n- **50일 이동평균선(SMA50)**: 67,648원 (중기 상승세 유지)\n- **RSI(상대강도지수)**: 50.51 (중립, 과매수/과매도 아님)\n- **거래량**: 1,010만주(